In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,52978
2,Huelva,Confirmados PDIA 14 días,4254
3,Huelva,Tasa PDIA 14 días,"828,9650603113978"
4,Huelva,Confirmados PDIA 7 días,2390
5,Huelva,Tasa PDIA 7 dias,"465,7326032308981"
6,Huelva,Total Confirmados,53187
7,Huelva,Curados,47180
8,Huelva,Fallecidos,426


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  52978.0


/tmp/ipykernel_3603/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  15030.0


/tmp/ipykernel_3603/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3603/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3603/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3603/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 173 personas en los últimos 7 días 

Un positivo PCR cada 98 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,52978.0,2390.0,4254.0,513170.0,465.732603,828.965060,415.0
Huelva-Costa,31324.0,1512.0,2568.0,289548.0,522.193211,886.899581,206.0
Condado-Campiña,16165.0,506.0,982.0,156231.0,323.879384,628.556432,114.0
Huelva (capital),15030.0,830.0,1461.0,143837.0,577.042068,1015.733087,103.0
Sierra de Huelva-Andévalo Central,5053.0,368.0,697.0,67391.0,546.066982,1034.262735,94.0
Valverde del Camino,1081.0,125.0,155.0,12750.0,980.392157,1215.686275,54.0
Aljaraque,1984.0,135.0,223.0,21474.0,628.667225,1038.465121,27.0
Ayamonte,2374.0,153.0,233.0,21104.0,724.981046,1104.056103,25.0
Moguer,2302.0,74.0,128.0,21867.0,338.409475,585.356931,19.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,59.0,33.0,34.0,614.0,5374.592834,5537.459283,0.0
Puerto Moral,21.0,3.0,10.0,278.0,1079.136691,3597.122302,0.0
Aracena,941.0,76.0,210.0,8255.0,920.654149,2543.912780,9.0
Fuenteheridos,36.0,13.0,18.0,709.0,1833.568406,2538.787024,2.0
Cumbres Mayores,196.0,14.0,38.0,1749.0,800.457404,2172.670097,1.0
Corteconcepción,57.0,1.0,7.0,536.0,186.567164,1305.970149,0.0
Cortegana,372.0,26.0,60.0,4602.0,564.971751,1303.780965,4.0
Valverde del Camino,1081.0,125.0,155.0,12750.0,980.392157,1215.686275,54.0
Almonaster la Real,116.0,8.0,21.0,1788.0,447.427293,1174.496644,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Corteconcepción,57.0,1.0,7.0,536.0,186.567164,1305.970149,0.0,0.142857
Campillo (El),121.0,4.0,20.0,2024.0,197.628458,988.142292,0.0,0.200000
Villarrasa,253.0,3.0,13.0,2211.0,135.685210,587.969245,1.0,0.230769
Granado (El),50.0,1.0,4.0,532.0,187.969925,751.879699,0.0,0.250000
Villanueva de las Cruces,27.0,1.0,4.0,387.0,258.397933,1033.591731,0.0,0.250000
Almendro (El),64.0,2.0,7.0,840.0,238.095238,833.333333,0.0,0.285714
Puerto Moral,21.0,3.0,10.0,278.0,1079.136691,3597.122302,0.0,0.300000
Manzanilla,176.0,7.0,23.0,2118.0,330.500472,1085.930123,0.0,0.304348
Aracena,941.0,76.0,210.0,8255.0,920.654149,2543.912780,9.0,0.361905
